In [36]:
### Illustration du biais de regularisation
### 26 septembre 2019
### Jeremy LHour

library("MASS")
library("glmnet")

# fonctions utilisateur
source("functions/DataSim.R") 

PostLasso <- function(X,y,nopen){
    #'@param nopen indices des variables a ne pas penaliser
    phi = rep(1,ncol(X)); phi[nopen] = 0
    cv.lasso = cv.glmnet(X,y, family="gaussian",alpha=1,penalty.factor=phi,nfolds=5)
    b.lasso = coef(cv.lasso); b.lasso = b.lasso[-1] # on enleve la constante
    s.hat = which(b.lasso != 0) # indices des variables actives parmi les X
    postlasso = lm(y ~ X[,s.hat])
    b.pl = coef(postlasso)

    return(b.pl)
}

PostStudent <- function(X,y,noppen,alpha=.05){
    #'@param nopen indices des variables a ne pas penaliser
    #'@param alpha niveau de significativite pour selection
    fullreg = lm(y ~ X)
    s.hat = summary(fullreg)$coef[,"Pr(>|t|)"] < alpha
    s.hat = s.hat[-1]
    s.hat = union(s.hat,nopen)
    shortreg = lm(y ~ X[,s.hat])
    b.ps = coef(shortreg)

    return(b.ps)
}

In [37]:
### Simulations
R = 100 # nb simulations
n = 2000 # sample size
p = 10 # nb variables
tau = 0 # vrai valeur de l'effet

set.seed(326)
Results = matrix(ncol=3, nrow=R)

In [39]:
t_start = Sys.time()

for(r in 1:R){
  ### GENERATE DATA
  data = suppressWarnings(DataSim(n=n,p=p,Ry=.3,Rd=.7,Intercept=F,a=tau))
  X = cbind(data$d,data$X); y = data$y
  
  ### METHOD 1: Modele complet
  full.reg = lm(y ~ X)

  ### METHOD 2: Selection Lasso
  lasso.reg = PostLasso(X,y,nopen=c(1))
  
  ### METHOD 3: Selection sur la stat de Student
  student.reg = PostStudent(X,y,noppen=c(1),alpha=.05)
  
  
  ### COLLECTING RESULTS
  Results[r,] = c(full.reg$coef[2],
                  lasso.reg[2],
                  student.reg[2])
}

print(Sys.time()-t_start)

head(Results)

Time difference of 1.246873 secs


0.4733416,0.4865791,1.528994
0.5476676,0.5537783,1.561721
0.5944681,0.5970645,1.606828
0.4784612,0.5230438,1.653076
0.5810590,0.6271770,1.695772
0.4083924,0.4467230,1.541817


In [24]:
### COMPUTE BIAS AND RMSE
StatDisplay = data.frame()
StatDisplay[1:3,"bias"] = apply(Results-a,2,mean)
StatDisplay[1:3,"RMSE"] = sqrt(apply((Results-a)^2,2,mean))
row.names(StatDisplay) = c("Naive","Immunized","Immunized, Cross-fitted")
print(StatDisplay)

### DRAW CHARTS
id = c(mapply(function(x) rep(x,R),1:3))
val = c(Results)-a
data_res = data.frame(val = val, model = id)

M = max(abs(quantile(Results,.01,na.rm=T)),abs(quantile(Results,.99,na.rm=T)))
lb = -1.1*M; ub = 1.1*M

get.plot <- function(data,modelS,title="A Title",s){
  plot_res <- ggplot(subset(data, (model==modelS)), aes(x=val)) + 
    geom_histogram(binwidth = .02, alpha=.5, position='identity',fill="steelblue", aes(y = ..density..)) +
    scale_x_continuous(limits=c(lb,ub), name="Treatment effect") +
    ggtitle(title) + 
    stat_function(fun = dnorm, args=list(mean=0, sd=s), colour="darkorchid3", size=1) +
    theme(plot.title = element_text(lineheight=.8, face="bold"),legend.position="none")
  return(plot_res)
} # plot func

pdf("plots/Immunized.pdf",width=14,height=4)
grid.arrange(get.plot(data_res,1,"Naive Post-Selec", mean(stdev)), get.plot(data_res,2,"Double-Selec", mean(stdev)), get.plot(data_res,3,"Double-Selec, Cross-fitting", mean(stdev)), ncol=3)
dev.off()

Warning message in c * gamma:
“Recycling array of length 1 in array-vector arithmetic is deprecated.
  Use c() or as.vector() instead.
”Warning message in c * b:
“Recycling array of length 1 in array-vector arithmetic is deprecated.
  Use c() or as.vector() instead.
”


Call:
lm(formula = y ~ X)

Residuals:
    Min      1Q  Median      3Q     Max 
-3.6823 -0.6406  0.0075  0.6506  3.2331 

Coefficients:
             Estimate Std. Error t value Pr(>|t|)    
(Intercept) -0.033320   0.071929  -0.463   0.6432    
X1           0.545330   0.061178   8.914   <2e-16 ***
X2          -0.700560   0.037144 -18.860   <2e-16 ***
X3           0.118896   0.060976   1.950   0.0513 .  
X4          -0.041923   0.029689  -1.412   0.1581    
X5           0.007144   0.064792   0.110   0.9122    
X6          -0.016198   0.053211  -0.304   0.7608    
X7           0.020552   0.052928   0.388   0.6978    
X8          -0.024606   0.049804  -0.494   0.6213    
X9           0.117041   0.061616   1.900   0.0576 .  
X10         -0.005923   0.035072  -0.169   0.8659    
X11          0.039704   0.057681   0.688   0.4913    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 1.01 on 1988 degrees of freedom
Multiple R-squared:  0.5155,	Adjusted